In [1]:
import numpy as np
import math
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
def print_tree(dtreehead):
    dtreehead.printselfdata()
    if(type(dtreehead.rightsplit)==int):
        return
    if(type(dtreehead.leftsplit)==int):
        return
    if(dtreehead.is_leaf==1):
        return
    print_tree(dtreehead.leftsplit)
    print_tree(dtreehead.rightsplit)
def entropy_calc(dtreehead):
    dtreehead.entropycalc()
    if(type(dtreehead.rightsplit)==int):
        return
    if(type(dtreehead.leftsplit)==int):
        return
    if(dtreehead.is_leaf==1):
        return
    entropy_calc(dtreehead.leftsplit)
    entropy_calc(dtreehead.rightsplit)
def splits(x_train, y_train, feature):
    split1x=[]
    split1y=[]
    split2x=[]
    split2y=[]
    for i in range(x_train.shape[0]):
        if x_train[i][feature]>=np.median(x_train[:,feature]):
            split1x.append(x_train[i])
            split1y.append(y_train[i])
        else:
            split2x.append(x_train[i])
            split2y.append(y_train[i])
    split1x=np.array(split1x)
    split1y=np.array(split1y)
    split2x=np.array(split2x)
    split2y=np.array(split2y)
    return split1x, split1y, split2x, split2y
def createtree(dtreehead):
    if(dtreehead.is_leaf==1):
        return
    info_gain=np.zeros(dtreehead.features.shape[0])
    curr=0
    for i in dtreehead.features:
        temp=np.delete(dtreehead.features,np.where(dtreehead.features==i))
        split1x,split1y,split2x,split2y=splits(dtreehead.x_data, dtreehead.y_data, i-1)
        left_split=node(split1x,split1y, temp)
        right_split=node(split2x, split2y, temp)
        left_split.entropycalc()
        right_split.entropycalc()
        leftratio=left_split.no_of_elements/dtreehead.no_of_elements
        rightratio=right_split.no_of_elements/dtreehead.no_of_elements
        num=(dtreehead.entropy-(leftratio*left_split.entropy)-(rightratio*right_split.entropy))
        den=((-1)*((leftratio*math.log2(leftratio))+(rightratio*math.log2(rightratio))))
        info_gain[np.where(dtreehead.features==i)]=num/den
    curr=np.argmax(info_gain)+1
    temp=np.delete(dtreehead.features, np.where(dtreehead.features==curr))
    dtreehead.feature=curr
    dtreehead.median=(np.median(dtreehead.x_data[:,curr-1]))
    split1x,split1y,split2x,split2y=splits(dtreehead.x_data, dtreehead.y_data, curr-1)
    if(split1x.shape[0]*split2x.shape[0]!=0):
        dtreehead.leftsplit=node(split1x,split1y, temp)
    if(split1x.shape[0]*split2x.shape[0]!=0):
        dtreehead.rightsplit=node(split2x, split2y, temp)
    if(dtreehead.is_leaf==1):
        return
    if(split1x.shape[0]*split2x.shape[0]!=0):
        createtree(dtreehead.leftsplit)
    if(split1x.shape[0]*split2x.shape[0]!=0):
        createtree(dtreehead.rightsplit)
class node:
    def __init__(self, x_train, y_train, features):
        self.x_data=x_train
        self.y_data=y_train
        self.cz=0
        self.co=0
        self.ct=0
        self.features=features
        self.no_of_elements=x_train.shape[0]
        self.leftsplit=0
        self.rightsplit=0
        self.feature=0
        self.median=0
        self.prob=[]
        if(len(features)==0 or len(set(y_train))==1):
            self.is_leaf=1
        else:
            self.is_leaf=0
        self.entropy=0
#         print(self.x_data, self.y_data)
    def entropycalc(self):
        for i in range(len(self.y_data)):
            if self.y_data[i]==1:
                self.co+=1;
            elif self.y_data[i]==2:
                self.ct+=1;
            else:
                self.cz+=1;
        self.counts=np.array([self.cz,self.co,self.ct])
        self.pz=self.cz/self.no_of_elements
        self.po=self.co/self.no_of_elements
        self.pt=self.ct/self.no_of_elements
        self.prob=np.array([self.pz,self.po,self.pt])
#         print(prob)
        for i in range(3):
            if(self.prob[i]!=0):
                self.entropy+=(self.prob[i]*math.log2(self.prob[i]))
        if(self.entropy!=0):
            self.entropy=self.entropy*(-1)
    def printselfdata(self):
        print(self.x_data.shape[0], self.entropy, self.counts, self.prob,self.feature, self. features,self.median)
def run(x_train, y_train):
    dtreehead=node(x_train, y_train, np.arange(1,x_train.shape[1]+1))
    createtree(dtreehead)
    return dtreehead
#     y_pred=predict(x_test,y_test)
#     print(confusion_matrix(y_test, y_test_pred))
def helper(x_testa, dtreehead):
    if(type(dtreehead.rightsplit)==int):
        return np.argmax(dtreehead.counts)
    if(type(dtreehead.leftsplit)==int):
        return np.argmax(dtreehead.counts)
    if(dtreehead.is_leaf==1):
        return np.argmax(dtreehead.counts)
    if(x_testa[dtreehead.feature-1]>=dtreehead.median):
        return helper(x_testa, dtreehead. leftsplit)
    return helper(x_testa, dtreehead.rightsplit)
def predict(x_test, dtreehead):
    y_pred=np.zeros(x_test.shape[0])
    for i in range(x_test.shape[0]):
        curr=helper(x_test[i],dtreehead)
        y_pred[i]=curr
    return y_pred
iris=datasets.load_iris()
x_train, x_test, y_train, y_test=train_test_split(iris.data, iris.target)
head=run(x_train, y_train)
entropy_calc(head)
print_tree(head)
y_pred=predict(x_test, head)
y_pred-y_test

112 1.5834545859901241 [38 35 39] [0.33928571 0.3125     0.34821429] 4 [1 2 3 4] 1.3
64 0.9652016987500656 [ 0 25 39] [0.       0.390625 0.609375] 3 [1 2 3] 5.1
33 0.19590927087360493 [ 0  1 32] [0.         0.03030303 0.96969697] 2 [1 2] 3.0
21 0.0 [ 0  0 21] [0. 0. 1.] 0 [1] 0
12 0.41381685030363374 [ 0  1 11] [0.         0.08333333 0.91666667] 1 [1] 6.4
8 0.0 [0 0 8] [0. 0. 1.] 0 [] 0
4 0.8112781244591328 [0 1 3] [0.   0.25 0.75] 0 [] 0
31 0.7706290693639405 [ 0 24  7] [0.         0.77419355 0.22580645] 2 [1 2] 2.9
18 0.5032583347756457 [ 0 16  2] [0.         0.88888889 0.11111111] 1 [1] 6.1
11 0.4394969869215134 [ 0 10  1] [0.         0.90909091 0.09090909] 0 [] 0
7 0.5916727785823275 [0 6 1] [0.         0.85714286 0.14285714] 0 [] 0
13 0.961236604722876 [0 8 5] [0.         0.61538462 0.38461538] 1 [1] 5.7
9 0.9910760598382222 [0 5 4] [0.         0.55555556 0.44444444] 0 [] 0
4 0.8112781244591328 [0 3 1] [0.   0.75 0.25] 0 [] 0
48 0.7382848661434506 [38 10  0] [0.79166667 0.20833333

array([ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0., -1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])